# CWE

In [ ]:
conda install xmltodict

In [1]:
import json
import os
import requests
import xmltodict
import zipfile

## 1. Pobieranie danych

In [44]:
CWE_FILE_URL = "https://cwe.mitre.org/data/xml/cwec_latest.xml.zip"
CWE_OUTPUT_DIR = "cwe_output"
FILE_DIR_NAME = "files"

CWE_OUTPUT_DIRPATH = os.path.join('files', CWE_OUTPUT_DIR)

# stałe dotyczące zipa
cwe_zip_output_filename = f'{CWE_FILE_URL.split("/")[-1].split(".")[0]}.zip'
CWE_ZIP_FILEPATH = os.path.join('files', CWE_OUTPUT_DIR, cwe_zip_output_filename)

# stałe dotyczące xml
XML_DICT_FILENAME = 'cwe.json'
XML_DICT_FILEPATH = os.path.join(CWE_OUTPUT_DIRPATH, XML_DICT_FILENAME)

### 1.1 pobieranie i zapisywanie zipa

In [45]:
data = requests.get(CWE_FILE_URL)
with open(CWE_ZIP_FILEPATH, "wb") as f:
    f.write(data.content)
    
print(CWE_ZIP_FILEPATH)

files\cwe_output\cwec_latest.zip


### 1.2 rozpakowywanie zipa

In [46]:
with zipfile.ZipFile(CWE_ZIP_FILEPATH, "r") as zip_ref:
    extracted_file_names = zip_ref.namelist()
    zip_ref.extractall(CWE_OUTPUT_DIRPATH)

# tylko jedne plik w zipie
extracted_file = extracted_file_names[0]
xml_filepath = os.path.join(CWE_OUTPUT_DIRPATH, extracted_file)
print(xml_filepath)

files\cwe_output\cwec_v4.9.xml


### 1.3 zamiana xml na jsona

In [47]:
xml_data = open(xml_filepath, "r", encoding="utf-8").read()
parsed_data = xmltodict.parse(xml_data)
xml_dict =  dict(parsed_data)

### 1.4 zapisywanie jsona

In [48]:
with open(XML_DICT_FILEPATH, "w", encoding="utf-8") as f:
    json.dump(xml_dict, f, indent=4)

### 1.5 Usuwanie zipa

In [49]:
os.remove(CWE_ZIP_FILEPATH)

### 1.6 Usuwanie pliku xml

In [31]:
# tu nie może być stała, bo to z pliku zipa nazwa pliku
os.remove(xml_filepath)

# 2. Parsowanie danych